In [1]:
from glob import glob
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import nibabel as nib
from nibabel.streamlines import load as load_trk
import dipy.tracking.streamline as dts
from skimage.transform import resize
from scipy.ndimage.morphology import binary_dilation

In [3]:
import keras
from keratin.networks import vgg16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [4]:
import bundlenet as bn

In [5]:
# Just choose the first one
atlas_img = nib.load('/home/ariel/data/volume/Acoustic_Radiation_L.nii.gz')

In [6]:
atlas_affine = atlas_img.affine
atlas_shape = atlas_img.shape

In [7]:
atlas_shape

(157, 189, 136)

In [8]:
group_dirs = glob('/home/ariel/data/tracks/*')

In [9]:
exclude = [op.join('/home/ariel/data/tracks/', g) for g in ["cerebellum", "brainstem", "cranial nerve"]]

In [10]:
group_dirs = [g for g in group_dirs if g not in exclude]

In [11]:
track_files_groups = [glob(op.join(d, '*.trk.gz')) for d in group_dirs]

In [12]:
track_files = []
_ = [track_files.extend(t) for t in track_files_groups]

In [13]:
n_streamlines = []
bundle_name = []
for fname in track_files:
    bundle_name.append(fname.split('/')[-1].split('bundles_')[-1].split('.trk')[0])
    streamlines = bn.read_sl(fname) 
    n_streamlines.append(len(streamlines))

In [14]:
np.max(n_streamlines), np.min(n_streamlines), len(n_streamlines)

(45747, 72, 45)

In [15]:
take_n_bundles = len(n_streamlines)

In [16]:
take_n_sl = np.min(n_streamlines)

In [ ]:
take_n_bundles = 45
take_n_sl = 70

In [ ]:
data_train, data_valid, data_test, labels_train, labels_valid, labels_test =\
    bn.partition_data(track_files, (300, 300, 300), take_n_bundles, take_n_sl)

/usr/local/lib/python3.4/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [ ]:
# data_train = np.zeros((np.int(np.round(take_n_bundles * take_n_sl * 0.6)),  100, 100, 1), dtype='float32')
# data_valid = np.zeros((np.int(np.round(take_n_bundles * take_n_sl * 0.2)),  100, 100, 1), dtype='float32')
# data_test = np.zeros((np.int(np.round(take_n_bundles * take_n_sl * 0.2)),  100, 100, 1), dtype='float32')

# labels_train = np.zeros(np.int(np.round(take_n_bundles * take_n_sl * 0.6)))
# labels_valid = np.zeros(np.int(np.round(take_n_bundles * take_n_sl * 0.2)))
# labels_test = np.zeros(np.int(np.round(take_n_bundles * take_n_sl * 0.2)))

# ii_train = 0
# ii_valid = 0
# ii_test = 0

# tract_id = 0
# for group_dir, group in zip(group_dirs, track_files):
#     if group_dir not in exclude:
#         for fname in group:
#             tract_id += 1
#             if tract_id > take_n_bundles: 
#                 break
#             streamlines = read_sl(fname, atlas_affine) 
#             choose_sl = streamlines[:take_n_sl]
#             # Shuffle them in case they are ordered somehow:
#             np.random.shuffle(choose_sl)
#             for sl_idx, sl in enumerate(choose_sl):
#                 assert np.all(sl > 0), "Negatory"
#                 projected = reduce_sl(sl)
#                 if sl_idx < (np.round(take_n_sl * 0.2)):
#                     data_test[ii_test] = projected
#                     labels_test[ii_test] = tract_id
#                     ii_test += 1
#                 elif sl_idx < (np.round(take_n_sl * 0.4)):
#                     data_valid[ii_valid] = projected
#                     labels_valid[ii_valid] = tract_id
#                     ii_valid += 1
#                 else:
#                     data_train[ii_train] = projected
#                     labels_train[ii_train] = tract_id
#                     ii_train += 1

In [ ]:
img_rows = 100
img_cols = 100
batch_size = 4
epochs = 5
num_classes = take_n_bundles
input_shape = (img_rows, img_cols, 1)

In [ ]:
labels_train = keras.utils.to_categorical(labels_train-1, num_classes)
labels_valid = keras.utils.to_categorical(labels_valid-1, num_classes)
labels_test  = keras.utils.to_categorical(labels_test-1, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
training = model.fit(data_train, labels_train,
                     batch_size=batch_size,
                     epochs=epochs,
                     verbose=1,
                     validation_data=(data_valid, labels_valid))

In [ ]:
score = model.evaluate(data_test, labels_test, verbose=0)

In [ ]:
print(score)

In [ ]:
fig = bn.plot_accuracy(training)